<a href="https://colab.research.google.com/github/msmsm104/TIL/blob/main/20220601/%ED%83%80%EC%9D%B4%ED%83%80%EB%8B%89_%EC%83%9D%EC%A1%B4%EC%9E%90_%EC%98%88%EC%B8%A1(Classification)_0601.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Source : https://www.kaggle.com/competitions/titanic

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Data Load

In [2]:
base_path = '/content/drive/MyDrive/Colab Notebooks/이어드림스쿨/기업강의(fastcampus)/인공지능 머신러닝(0530)/03. Classification/data/'

train = pd.read_csv(base_path + 'train.csv')
test = pd.read_csv(base_path + 'test.csv')
submission = pd.read_csv(base_path + 'gender_submission.csv')



In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


---

## Data Preprocessing

    1. 결측치 처리
    2. feature selection(분석에 사용하지 않는 column제거)

In [8]:
train.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [9]:
## 결측치가 가장 적은 Embarked부터 처리해주자. 

train['Embarked'].value_counts() 

# 1. 최빈값으로 채우는 방법
# 2. 평균값으로 채우는 방법
# 3. 비슷한 속성의 그룹과 통일시키는 방법

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [11]:
# 3. 결측치 데이터의 속성을 알아보자

train[train['Embarked'].isna()]


## 이름은 다르지만 티켓 번호와 cabin이 동일하다. 뭔가 연관성이 있는 두사람임을 추측가능하다.

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
61,62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN
829,830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN


In [14]:
train[(train['Ticket'] == '113572') | (train['Cabin'] == 'B28')]

## 해당 특성으로 동일한 승객이 없으므로 다른 특성으로 비슷한 군집을 알아보자.

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
61,62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN
829,830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN


In [16]:
## Pclass 가 1, Sex가 female인 집단.

train[(train['Pclass'] == 1) & (train['Sex'] == 'female')]['Embarked'].value_counts()

## Pclass 와 Sex를 기준으로 Embarked의 비율을 보았을때 역시 S가 최빈값이므로, 결측값을 S로 채워준다.

S    48
C    43
Q     1
Name: Embarked, dtype: int64

In [20]:
train['Embarked'] = train['Embarked'].fillna('S')

In [21]:
train.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64

In [22]:
train[(train['Pclass'] == 1) & (train['Sex'] == 'female')]['Embarked'].value_counts()
## 결측값이 잘 채워졌음을 확인할 수 있다.

S    50
C    43
Q     1
Name: Embarked, dtype: int64

In [25]:
## Age column의 결측치 처리 (평균, 최빈, 추정가능 데이터로 처리가능..)

## 여기서는 평균값으로 처리


train['Age'] = train['Age'].fillna(train['Age'].mean())

In [27]:
train.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64

In [29]:
## Cabin column의 경우 생존자 예측과 큰 상관관계가 없다고 판단되므로 결측치를 따로 처리해주지 않는다.

## 이번에는 생존자 예측과 관련없는 column을 제거해보자. 
# (PassengerId, Name, Ticket, Cabin)

train = train.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'])
train

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.000000,1,0,7.2500,S
1,1,1,female,38.000000,1,0,71.2833,C
2,1,3,female,26.000000,0,0,7.9250,S
3,1,1,female,35.000000,1,0,53.1000,S
4,0,3,male,35.000000,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.000000,0,0,13.0000,S
887,1,1,female,19.000000,0,0,30.0000,S
888,0,3,female,29.699118,1,2,23.4500,S
889,1,1,male,26.000000,0,0,30.0000,C


In [30]:
train.isna().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

---

## Feature Engineering

    1. Categorical feature encoding
    2. Normalization

In [31]:
train.head(2)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C


In [32]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       891 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  891 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


In [37]:
## Categorical feature --> One-Hot Encoding, Ordinal Encoding

train_OHE = pd.get_dummies(train, columns = ['Sex', 'Embarked'], drop_first=True)
train_OHE.head(2)

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S
0,0,3,22.0,1,0,7.2500,1,0,1
1,1,1,38.0,1,0,71.2833,0,0,0


In [42]:
X = train_OHE.drop(columns = 'Survived') # input matrix
y = train_OHE['Survived'] # target vector
display(X.head(2))
display(y)

,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S
0,3,22.0,1,0,7.2500,1,0,1
1,1,38.0,1,0,71.2833,0,0,0


0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [46]:
## Normalization (정규화) - 0~1사이의 값으로

### x - min(X) / max(X) - min(X)

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
# X_scaled


In [48]:
X = pd.DataFrame(data=X_scaled, columns=X.columns)
X

,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S
0,1.0,0.271174,0.125,0.000000,0.014151,1.0,0.0,1.0
1,0.0,0.472229,0.125,0.000000,0.139136,0.0,0.0,0.0
2,1.0,0.321438,0.000,0.000000,0.015469,0.0,0.0,1.0
3,0.0,0.434531,0.125,0.000000,0.103644,0.0,0.0,1.0
4,1.0,0.434531,0.000,0.000000,0.015713,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...
886,0.5,0.334004,0.000,0.000000,0.025374,1.0,0.0,1.0
887,0.0,0.233476,0.000,0.000000,0.058556,0.0,0.0,1.0
888,1.0,0.367921,0.125,0.333333,0.045771,0.0,0.0,1.0
889,0.0,0.321438,0.000,0.000000,0.058556,1.0,0.0,0.0


---

## Training

In [49]:
## classifier model
from sklearn.linear_model import SGDClassifier      # 1. Linear Classifier
from sklearn.linear_model import LogisticRegression # 2. Logistic Regression
from sklearn.tree import DecisionTreeClassifier     # 3. Decision Tree(CART)
from sklearn.ensemble import RandomForestClassifier # 4. Random Forest

## 평가 지표
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

    테스트 데이터는 csv형태로 따로 제공되고 있으므로 해당 train데이터에서 validation data를 따로 분리해서 학습성능과 validation 데이터에 대한 성능을 평가해보자.

In [56]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = \
train_test_split(X, y, test_size=0.1, random_state=100)

In [82]:
## row = model, column = 평가지표
accuracy_lst = []
precision_lst = []
recall_lst = []
f1_lst = []

def training(model_):
    clf = model_
    clf.fit(X_train, y_train)

    ## 학습결과의 성능.
    pred_train = clf.predict(X_train)
    pred_val = clf.predict(X_val)

    print(f"=====Accuracy=====")
    print(f"Accuracy for Training: {accuracy_score(y_train, pred_train):.4f}")
    print(f"Accuracy for Validation: {accuracy_score(y_val, pred_val):.4f}")
    
    print(f"=====Precision=====")
    print(f"Precision for Training: {precision_score(y_train, pred_train):.4f}")
    print(f"Precision for Validation: {precision_score(y_val, pred_val):.4f}")
    print(f"=====Recall=====")
    print(f"Recall for Training: {recall_score(y_train, pred_train):.4f}")
    print(f"Recall for Validation: {recall_score(y_val, pred_val):.4f}")
    print(f"=====F1_score=====")
    print(f"F1-score for Training: {f1_score(y_train, pred_train):.4f}")
    print(f"F1-score for Validation: {f1_score(y_val, pred_val):.4f}")

In [83]:
training(SGDClassifier())

=====Accuracy=====
Accuracy for Training: 0.8140
Accuracy for Validation: 0.7778
=====Precision=====
Precision for Training: 0.8153
Precision for Validation: 0.7857
=====Recall=====
Recall for Training: 0.6634
Recall for Validation: 0.6111
=====F1_score=====
F1-score for Training: 0.7315
F1-score for Validation: 0.6875


In [84]:
training(LogisticRegression())

=====Accuracy=====
Accuracy for Training: 0.8015
Accuracy for Validation: 0.7667
=====Precision=====
Precision for Training: 0.7543
Precision for Validation: 0.7586
=====Recall=====
Recall for Training: 0.7124
Recall for Validation: 0.6111
=====F1_score=====
F1-score for Training: 0.7328
F1-score for Validation: 0.6769


In [85]:
training(DecisionTreeClassifier())

=====Accuracy=====
Accuracy for Training: 0.9863
Accuracy for Validation: 0.7444
=====Precision=====
Precision for Training: 0.9966
Precision for Validation: 0.6857
=====Recall=====
Recall for Training: 0.9673
Recall for Validation: 0.6667
=====F1_score=====
F1-score for Training: 0.9818
F1-score for Validation: 0.6761


In [86]:
training(RandomForestClassifier())

=====Accuracy=====
Accuracy for Training: 0.9863
Accuracy for Validation: 0.8333
=====Precision=====
Precision for Training: 0.9900
Precision for Validation: 0.8182
=====Recall=====
Recall for Training: 0.9739
Recall for Validation: 0.7500
=====F1_score=====
F1-score for Training: 0.9819
F1-score for Validation: 0.7826


### Train data에 대한 학습, 후 평가 

    1. linear < CART < RandomForest 순으로 모델의 성능이 향상됨을 알 수 잇다.
    2. tree 계열의 모델에서 성능이 높게 나오지만 validation data를 기준으로 보앗을때 training data의 결과값과 차이가 잇으므로 overfitting을 의심해볼 수 잇다.

---

## Test (predict)
    - 주어진 test 데이터를 기반으로 모델의 성능을 평가해보자

In [89]:
## test data도 train data에 적용한 것과 동일한 feature engineering을 적용해 주어야한다.

# 1. 결측치 처리(age의 경우 train data의 age 평균값으로 넣어주어야한다.)
# 2. Normalization (scaling- minmax)

test.head(2)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S


In [91]:
## 우선 학습에 필요하지 않은 column부터 제거

test = test.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'])


In [92]:
# 결측치 확인
test.isna().sum()

Pclass       0
Sex          0
Age         86
SibSp        0
Parch        0
Fare         1
Embarked     0
dtype: int64

In [93]:
train

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.000000,1,0,7.2500,S
1,1,1,female,38.000000,1,0,71.2833,C
2,1,3,female,26.000000,0,0,7.9250,S
3,1,1,female,35.000000,1,0,53.1000,S
4,0,3,male,35.000000,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.000000,0,0,13.0000,S
887,1,1,female,19.000000,0,0,30.0000,S
888,0,3,female,29.699118,1,2,23.4500,S
889,1,1,male,26.000000,0,0,30.0000,C


In [94]:
test['Age'] = test['Age'].fillna(29.699118)

In [96]:
test.isna().sum()

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        1
Embarked    0
dtype: int64

In [97]:
## Fare의 경우도 마찬가지로 train Fare의 평균값으로 채워준다.

test['Fare'] = test['Fare'].fillna(train['Fare'].mean())

In [98]:
test.isna().sum()

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [102]:
# categorical feature --> One-Hot Encoding

test_OHE = pd.get_dummies(test, columns=['Sex', 'Embarked'], drop_first=True)
test_OHE

,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S
0,3,34.500000,0,0,7.8292,1,1,0
1,3,47.000000,1,0,7.0000,0,0,1
2,2,62.000000,0,0,9.6875,1,1,0
3,3,27.000000,0,0,8.6625,1,0,1
4,3,22.000000,1,1,12.2875,0,0,1
...,...,...,...,...,...,...,...,...
413,3,29.699118,0,0,8.0500,1,0,1
414,1,39.000000,0,0,108.9000,0,0,0
415,3,38.500000,0,0,7.2500,1,0,1
416,3,29.699118,0,0,8.0500,1,0,1


In [104]:
## Normalization - Train의 data기준에 맞춰야하기때문에 scaler model을 다시 fit하지 않는다.

scaled_data = scaler.transform(test_OHE)

In [107]:
test_OHE = pd.DataFrame(data=scaled_data, columns=test_OHE.columns)

In [127]:
## train, validation data를 기반으로 보았을때 가장 성능이 좋았을 random forest를 기준으로 성능을 평가해보자

model = RandomForestClassifier(criterion='gini', 
                            n_estimators=700,
                             min_samples_split=10,
                             min_samples_leaf=1,
                             max_features='auto',
                             oob_score=True,
                             random_state=1,
                             n_jobs=-1)

In [128]:
model.fit(X_train, y_train)

RandomForestClassifier(min_samples_split=10, n_estimators=700, n_jobs=-1,
                       oob_score=True, random_state=1)

In [125]:
model.feature_importances_[0]

0

In [132]:
pred = model.predict(test_OHE)

# accuracy_score(y_val, pred)

In [133]:
submission['Survived'] = pred

In [134]:
submission

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [135]:
submission.to_csv(base_path + 'submission_3.csv', index=False)

---